In [320]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
from io import BytesIO
from PIL import Image
import cv2
import re

## Самый эффективный способ усреднять - брать среднее квадратов меток
. 
Трешхолдить, брать максимум - все дает рез-ты сильно хуже (на 4-5 сотых), чем просто среднее. А среднее на 0.1 хуже среднего квадратов


В сабмитах порядок был такой:
 * просто среднее 0.623684675521249
 * максимум  0.669751155620737
 * медиана после трешхолда по 0.7  0.657896216086618

In [321]:
df = pd.read_csv("Train_fixed.csv")
df["img_name"] = df["UID"].apply(lambda x: ".".join([x, "jpeg"]))
df.head()

,UID,growth_stage,label_quality,img_name
0,nwuCTQKz,4,2,nwuCTQKz.jpeg
1,NVGUOzWu,1,1,NVGUOzWu.jpeg
2,1IYVvHwA,1,1,1IYVvHwA.jpeg
3,8OEdWwmU,1,1,8OEdWwmU.jpeg
4,oc5iO04j,1,1,oc5iO04j.jpeg


In [322]:
df.shape

(10695, 4)

 * rmse 0.5287153386994643  file full_pred_b3_16.csv after 39 epochs
 * rmse 0.5095805723408371  file full_pred_b1_16.csv  public lb 0.58939475057617 after 39 epochs 61=39.chkp
 объединив 2 предсказания выше получается rmse 0.49851644668196105
 
 * rmse file full_pred_b1_60_16.csv after 59 epochs  public lb 0.599793856286765

 * rmse 0.5404515107599817 file middle_small_17.csv 11=34.chkp 
 

In [323]:
subm_df = pd.read_csv("full_data_rn34_01.csv")
columns = subm_df.columns
print(subm_df.shape)
pred = []
for g in subm_df.groupby("UID"):
    p = g[1]["pred"]**2
    #print(g[1]["UID"].unique()[0])
    pred.append([g[1]["UID"].unique()[0], g[1]["img_name"].unique()[0], np.max([np.min([np.sqrt(p.mean()), 6]),0])+1])
subm_df = pd.DataFrame(data = pred, columns = columns)

subm_df.head()

(160425, 3)


,UID,img_name,pred
0,01AJUu7l,01AJUu7l.jpeg,3.890042
1,01B4CV3K,01B4CV3K.jpeg,2.192580
2,02R9pCXz,02R9pCXz.jpeg,5.854731
3,02XgtYwT,02XgtYwT.jpeg,6.756191
4,02hINknf,02hINknf.jpeg,6.382838


In [324]:
#subm_df = pd.read_csv("pred_rn50_24.csv")

In [325]:
def RMSE(pred, y):
    sq = (pred - y) * (pred - y)
    #print(sq.hist())
    rmse = np.sqrt(np.mean(sq, axis=0)) 
    return rmse
#RMSE(np.array([0.,1.,2.,3.]),np.array([0.,1.,2.,2.]))

In [326]:
#subm_df=pd.read_csv("large_rn34.csv")

In [327]:
subm_df = subm_df.merge(df[["UID", "growth_stage", "label_quality"]], on="UID")
subm_df.head(20)

,UID,img_name,pred,growth_stage,label_quality
0,01AJUu7l,01AJUu7l.jpeg,3.890042,7,1
1,01B4CV3K,01B4CV3K.jpeg,2.192580,2,2
2,02R9pCXz,02R9pCXz.jpeg,5.854731,7,1
3,02XgtYwT,02XgtYwT.jpeg,6.756191,7,2
4,02hINknf,02hINknf.jpeg,6.382838,5,1
5,03suVk46,03suVk46.jpeg,3.771892,2,1
6,03vQEfIz,03vQEfIz.jpeg,4.088024,1,1
7,04J3L9gf,04J3L9gf.jpeg,4.741621,6,1
8,04Opjmbk,04Opjmbk.jpeg,3.257702,3,2
9,053h7FH8,053h7FH8.jpeg,4.114574,4,1


In [328]:
subm_df["UID"].shape

(10695,)

In [329]:
df[df["UID"].isin(subm_df["UID"])].shape

(10695, 4)

In [330]:
subm_df[subm_df["UID"].isin(df["UID"])].shape

(10695, 5)

In [331]:
subm_df.duplicated(subset="UID").sum()

0

In [332]:
RMSE(subm_df[subm_df["label_quality"] == 2]["pred"], subm_df[subm_df["label_quality"] == 2]["growth_stage"])

0.32792465030845547

In [333]:
gs = []
pred = []
for r in subm_df[subm_df["label_quality"] == 2].iterrows():
    #print(r[1]["pred"])
    pred.extend([r[1]["pred"]])
    gs.extend(df[df["UID"] == r[1]["UID"]]["growth_stage"].to_numpy())
#print(gs, pred)
RMSE(np.array(gs), np.array(pred))

0.32792465030845536

In [334]:
0.4615597475340819
train_rn34_middle_22.csv  0.41800178754143724
middle rn50 0.4307019231117651
middle rn34 0.3402762293168213
small b1 0.473485872461246
small b1 train_b1_28.csv 0.44295648456503794  20 эпох верхние слои
train_b1_28_1.csv 0.4465984363380721 + 20 ''
large rn34 0.4377987754014158
large rn34 - первая утренняя попытка 0.3709703573172524
large rn34 на части данных 15 эпох 0.40300708160443405
то же на полных данных 0.4657637965825579

еще поучили rn34 0.3794301894500912

SyntaxError: invalid syntax (<ipython-input-334-71301ddb6680>, line 2)

In [ ]:
subm_df.sort_values(by="UID")["growth_stage"] = df.sort_values(by="UID")[df["UID"].isin(subm_df["UID"])]["growth_stage"]
#subm_df= subm_df.dropna()
RMSE(subm_df["growth_stage"], subm_df["pred"])

In [ ]:
subm_df[subm_df["ground_truth"] - subm_df["pred"] == 3].head(20)

In [ ]:
subm_df.loc[179]

In [ ]:
subm_df["p1"] = subm_df["1_data"].apply(lambda x: 1 if x>0.7 else None) * subm_df["1"]
subm_df["p2"] = subm_df["2_data"].apply(lambda x: 1 if x>0.7 else None) * subm_df["2"]
subm_df["p3"] = subm_df["3_data"].apply(lambda x: 1 if x>0.7 else None) * subm_df["3"]
subm_df["p4"] = subm_df["4_data"].apply(lambda x: 1 if x>0.7 else None) * subm_df["4"]
subm_df["p5"] = subm_df["5_data"].apply(lambda x: 1 if x>0.7 else None) * subm_df["5"]
subm_df.head(10)

In [ ]:
subm_df.shape

In [ ]:
df["UID"].to_numpy()

In [ ]:
new_arr = []
for i in subm_df.iterrows():
     if i[1]["UID"] not in df["UID"].to_numpy():
            new_arr.append(i[1])
new_df = pd.DataFrame(data = new_arr, columns = subm_df.columns)
new_df.head()

In [ ]:
new_df["pred"] = round(new_df[["p1", "p2", "p3", "p4", "p5"]].median(axis=1, skipna=True)) + 1

In [ ]:
new_df["pred"].hist()

In [ ]:
new_df[120:140]

In [ ]:
def gt2(x):
    #print(x.shape)
    n = np.argmax(x[:5], axis=0)
    return x[5+n]
new_df["gt2"] = new_df[["1_data","2_data","3_data","4_data","5_data",
                        "1", "2", "3", "4", "5"]].apply(lambda x: gt2(x) + 1, axis=1)
new_df.head(20)

In [ ]:
new_df[new_df["pred"] != new_df["gt2"]]
RMSE(new_df["pred"], new_df["gt2"])

In [ ]:
new_df[new_df["pred"] != new_df["growth_stage"]]
RMSE(new_df["pred"], new_df["growth_stage"])

In [ ]:
new_df[new_df["gt2"] != new_df["growth_stage"]]
RMSE(new_df["gt2"], new_df["growth_stage"])

In [ ]:
RMSE(subm1["growth_stage"], new_df["growth_stage"])

In [ ]:
new_df[["UID", "gt2"]]

In [ ]:
submission = pd.DataFrame(data = new_df[["UID", "growth_stage"]].to_numpy(), columns=["UID", "growth_stage"])  
submission.to_csv("submission_1109_2.csv", index=False)

In [ ]:
submission.head()

In [ ]:
submission.shape

In [ ]:
subm_df[subm_df["rmse"] > 8].head(20)

In [ ]:
subm1.sort_values("UID").head(20)

In [ ]:
new_df.sort_values("UID").head(20)

In [ ]:
subm_df["rmse"] = (subm_df["ground_truth"] - subm_df["pred"]) * (subm_df["ground_truth"] - subm_df["pred"])
np.sqrt(np.mean(subm_df["rmse"]))

In [ ]:
1.351383755217667

In [ ]:
subm_df.sort_values("img_name").head(20)

In [ ]:
subm1 = pd.read_csv("submission_1109.csv")
for i in subm1.iterrows():
    if new_df[new_df["UID"] == i[1]["UID"]].shape[0] == 0:
        print(i)
    #print(new_df[new_df["UID"] == i[1]["UID"]]["growth_stage"].to_numpy()[0])
    if new_df[new_df["UID"] == i[1]["UID"]]["growth_stage"].to_numpy()[0] != i[1]["growth_stage"]:
        print(new_df[new_df["UID"] == i[1]["UID"]])